# 8 sections(cells) of codes are prepared for different tasks(objectives) in open campus job. 
### These 8 sections are all error-free codes. Substitution is required in 'dir' to be viable in your case. High-performance rewrite may be needed to deal with big data or heavy files.

In summary, all 8 cells aims for extract some information e.g. date, time, longitude, latidute, filename from CTD files, applying to different-format & different naming CTD data. These slight discrepancies cause major changes in codes.<br>
- 1st one: trying to recognize/out the info from pdf scanned from paper documents. The recognization effectiveness is worrysome.<br>
- 2nd one: trying to sepatate sheets into sole excel files from excel files having multiple sheets. Codes are effective.<br>
- 3rd one: trying to extract station name, lat, lon etc. and put them into excel. Codes are effective.<br>
- 4th one: a auxiliary cell of 3rd one.<br>
- 5th one: second auxiliary cell of 3rd one.<br>
- 6th one: the motivation is same as 3rd one. Due to metadata has different format, the codes are far from the same.<br>
- 7th one: extracting these info from a questionnaire.<br>
- 8th one: extracting these info from a pretreated file.

In [ ]:
# Trying Optical Character Recognition (OCR)
# reading the text from scanned pdf
import pytesseract # OCR tool for python
from pdf2image import convert_from_path
import xlsxwriter # excel-related module
from tqdm import tqdm # arabic word meaning 'progress'

images=convert_from_path('[your path]on campus job\\test.pdf')
excel=xlsxwriter.Workbook('[your path]on campus job\\Book1.xlsx')
sheet=excel.add_worksheet()

for i in tqdm(range(10)):
    image=images[i]
    text=pytesseract.image_to_string(image,lang='jpn')
    column=0
    for item in text.splitlines():
        sheet.write(i,column, item)
        column+=1

excel.close() #11mins, low efficient

In [ ]:
# separating sheets into sole files from excel files
# each excel file has many sheets.
# *** is for personal privacy
import pandas as pd
import os #operating system 
from glob import glob # pathname pattern expansion
from tqdm.auto import tqdm # arabic word meaning 'progress'

path='[your path]\Otsuchi Bay CTD_Hideki *** (2011-2023)\\'
folders=sorted(glob(path+'2*'))
for k in tqdm(range(len(folders))):
    files=os.listdir(folders[k])
    for j in range(len(files)):
        excel=pd.ExcelFile(folders[k]+'\\'+files[j])
        i=0 # in some excel files, first sheet is the location info
        if excel.sheet_names[i]!='Location':
            df=pd.read_excel(folders[k]+'\\'+files[j],sheet_name=i)
            name=files[j][:-5]+excel.sheet_names[i]+'.csv'
            new=name.replace(" ","") #string method
            new2=new.replace("'","p") #string method
            df.to_csv(path+'Separated files\\'+new2)
        for i in range(1,len(excel.sheet_names)):
            df=pd.read_excel(folders[k]+'\\'+files[j],sheet_name=i)
            name=files[j][:-5]+excel.sheet_names[i]+'.csv'
            new=name.replace(" ","")
            new2=new.replace("'","p")
            df.to_csv(path+'Separated files\\'+new2)


In [ ]:
# extracting the station name, latitude, longitude, and writing them into excel

import pandas as pd
import os
import xlsxwriter
path='[your path]\Otsuchi Bay CTD_Hideki *** (2011-2023)\\'
files=os.listdir(path+"Separated files")
df=pd.read_excel(path+'Summary of station name and location in Otsuchi Bay.xlsx')
df.iloc[5,0]='1p';df.iloc[7,0]='2p'
df=df.astype({'Unnamed: 0':'string'}) # convert string-int mix into string

book=xlsxwriter.Workbook(path+'x.xlsx')
sh=book.add_worksheet()

for i in range(len(files)):
    try:
        if files[i].find('Ex.1')!=-1:
            SN='Ex.1'
        elif files[i].find('Ex1')!=-1:
            SN='Ex1'
        elif files[i].find('Sta.St2')!=-1:
            SN='2'
        else:
            SN=files[i].split('.')[-2] #station name
            if SN.find('(')!=-1: # some stations have multiple samplings
                SN=SN[:-3]
            if len(SN)>4: # some long string
                SN=SN[-2:]
            if SN.find('(採泥)')!=-1:# kanji undetectable
                SN=SN[:-4]
            if SN.find('_採泥')!=-1:
                SN=SN[:-3]
            
        Df=df.loc[df.iloc[:,0]==SN] # match the station
        lon=df.iloc[1,2]+str(Df.iloc[0,2])
        lat=df.iloc[1,1]+str(Df.iloc[0,1]);lat=lat.replace(" ","")
        Data=pd.read_csv(path+"Separated files\\"+files[i])# open the STD file
        date=Data.iloc[11,1][-10:]
        time=Data.iloc[12,1][-8:]
        if date.find('/')==-1: # -1 means the character not existed; data in different format
            date=Data.iloc[13,1][-19:-9]
            time=Data.iloc[13,1][-8:]
        
        sh.write(i,0,date);sh.write(i,1,time);sh.write(i,2,lat);sh.write(i,3,lon)
    except:sh.write(i,0,files[i]);continue
book.close() # it is necessary


In [ ]:
# Supplementary code for # NO.3 to correct the 'Time' column in most files in 2011
# the original time was the same for those files in one date, perhaps due to the old device system (personal ideas)
import pandas as pd
import os
import xlsxwriter
path='[your path]'
files=os.listdir(path+"Separated files")
List=[]
for i in files:
    if i[0:9]=='27May2011' or i[0:10]=='25July2011' or (i[0:15]=='20September2011') & (i[15:18]!='(2)') :
        List.append(i)

book=xlsxwriter.Workbook(path+'x.xlsx')
sh=book.add_worksheet()
for j,k in zip(List,range(len(List))):
    Data=pd.read_csv(path+"Separated files\\"+j)
    sh.write(k,0,Data.iloc[25,3])
book.close()

In [ ]:
# another supplementary code for # NO.3 to match the filename with sampling time.
# 2 ways can solve this problem: the first one is match the filename with time directly, which means another frame of code is necessary. the second one based on # NO.3, but it needs many munnual operation in the latter steps.
import os
import xlsxwriter
import numpy as np
path='C:\\Users\\***\\Downloads\\Otsuchi Bay CTD_*** (2011-2023)\\Otsuchi Bay CTD_*** (2011-2023)\\'
files=os.listdir(path+"Separated files")
Str=[]
Month=(('dummy','January','February','March','April','May','June','July','August','September','October','November','December'))
for i in range(len(files)):
    if files[i][0:8].isdigit()==True:
        Str.append(files[i][0:8]+files[i].split('.')[1])
    else:
        # due to the complexity of file name, 2 computations are needed.
        index=files[i].find('.')
        NewF=files[i][0:index]
        index2=NewF.find('201')
        NewF2=NewF[0:index2+4]
        index3=np.where(np.array(Month)==NewF2[2:index2])[0][0]
        Str.append(NewF2[index2:index2+4]+'%02d' % index3+NewF2[0:2]+files[i].split('.')[1])

Index=np.argsort(Str)
book=xlsxwriter.Workbook(path+'x.xlsx')
sh=book.add_worksheet()
for j in range(len(Index)):
    sh.write(j,0,files[Index[j]]),sh.write(j,1,Str[Index[j]])
book.close()

In [ ]:
# extracting the data, time, latitude, longitude, station, cruise number from multi-sheet excels to a single sheet
#KT: Tansei Maru; KK: Taisan Kaiyou Maru; KS: Shinsei Maru
# this code is prototype, further improvment is required to accomplish large dataset
# the core code is not hard to compile, while needing many minor adjustments

import pandas as pd # because the data is not so complicated, so user-friendly pandas module is used here.
import xlsxwriter
import os
path='[your path]\\Otsuchi Bay CTD_*** (2011-2023)\\Teams\\'
path2='[your path]\\Otsuchi Bay CTD_*** (2011-2023)\\'
files=os.listdir(path)
book=xlsxwriter.Workbook(path2+'x.xlsx')
sh=book.add_worksheet()
mon=('','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec') #first one is dummy data

row=0
for i in range(len(files)): #len(files)=25
    index1=1;index2=2;index3=3;index4=4
    if i ==0:            # this file has no station name in header
        index1=0;index2=1;index3=2;index4=3
    if i in range(7,21): # these files have extra depth data in header
        index1=2;index2=3;index3=4;index4=5
    cruise=files[i][0:8] #extracing the cruise name
    Data=pd.ExcelFile(path+files[i])
    count=len(Data.sheet_names) # counting how many sheets in one excel
    for j in range(2,count): #every first 2 sheets are map and locations
        #if Data.sheet_names[j].isascii()==True: # several ways to confirm whether the string is kanji or letter, actually every first 2 sheets are not CTD data
        data=pd.read_excel(path+files[i],sheet_name=j,header=None)
        if data.empty==True: # one sheet in excel (original data) is empty
            continue
        station=Data.sheet_names[j] #extracting the station name 
        try:lat=data.iloc[index1,0].split(':')[1].strip() # extracting the latitude
        except:
            Lat=data.iloc[index1,0].split('=')[1].replace(' ','') #splitted by colon and beginning space removed
            lat=Lat[:2]+'-'+Lat[2:]
        try:lon=data.iloc[index2,0].split(':')[1].strip() # extracting the longitude
        except:
            Lon=data.iloc[index2,0].split('=')[1].replace(' ','') 
            lon=Lon[:3]+'-'+Lon[3:]
        try:
            date=data.iloc[index3,0].split(':')[1].strip() #extracting the date
            time=data.iloc[index4,0].partition(':')[2].strip() #extracting the time
        except:
            UTC=data.iloc[index3,0].split('=')[1].replace(' ','')
            Date=UTC[:-8]
            date=Date[-4:]+'/'+str(mon.index(Date[0:3]))+'/'+Date[3:5] # formatting the date
            time=UTC[-8:]

        sh.write(row,0,date);sh.write(row,1,time);sh.write(row,2,lat);sh.write(row,3,lon);sh.write(row,4,station);sh.write(row,5,cruise) # write into the sheet
        row+=1
    print(f'done {i+1} time(s)')
    
book.close()

In [ ]:
# extract date, time, longitude, latitude into an excel
import pandas as pd
import xlsxwriter

book=xlsxwriter.Workbook('C:\\Users\\***\\Downloads\\Otsuchi Bay CTD_*** (2011-2023)\\x.xlsx')
sh=book.add_worksheet()
D=pd.ExcelFile('C:\\Users\\***\\Downloads\\F\\大槌CTDまとめ2013まで - N K.xlsx')
for i in range(len(D.sheet_names)):
    data=pd.read_excel('C:\\Users\\***\\Downloads\\F\\大槌CTDまとめ2013まで - N K.xlsx',sheet_name=i)
    sh.write(i,0,data.iloc[14,0][-10:]);sh.write(i,1,data.iloc[15,0][-8:])
    sh.write(i,2,str(data.iloc[0,2])+'N'+str(data.iloc[0,3]))
    sh.write(i,3,str(data.iloc[0,4])+'E'+str(data.iloc[0,5]))
    sh.write(i,4,'大槌CTDまとめ2013まで - N K.xlsx-'+D.sheet_names[i])
book.close()

In [ ]:
# extracting date, time, longitude, latittude into an excel (Object is different from last cell but can be combined into one)
import pandas as pd
import xlsxwriter
import math

book=xlsxwriter.Workbook('C:\\Users\\***\\Downloads\\Otsuchi Bay CTD_*** (2011-2023)\\x.xlsx')
sh=book.add_worksheet()
data=pd.read_excel('C:\\Users\\***\Downloads\\F\\CTD_itoh_metadata_2013-2016 - ***.xlsx')
for i in range(48):
    Str=data.iloc[i,2].split('.')
    sh.write(i,0,Str[0]+'/%02d'%int(Str[1])+'/%02d'%int(Str[2]))
    sh.write(i,1,str(data.iloc[i,3]))
    if math.isnan(data.iloc[i,6])==True:
        data.iloc[i,6]=0;data.iloc[i,9]=0
    sh.write(i,2,str(data.iloc[i,4])+'E'+str(data.iloc[i,5]+data.iloc[i,6]*0.01))
    sh.write(i,3,str(data.iloc[i,7])+'E'+str(data.iloc[i,8]+data.iloc[i,9]*0.01))
    sh.write(i,4,data.iloc[i,1])
book.close()
